In [1]:
# 데이터 불러오기

import pandas as pd
import numpy as np
import time
import warnings

warnings.filterwarnings(action='ignore')


print('종목 차트정보 가져오는 중...')
dailychart = pd.read_csv('dailychart.csv')
dailychart['date'] = pd.to_datetime(dailychart['date'], format='%Y%m%d')


# 날짜 데이터프레임 생성
df_date = pd.DataFrame(dailychart['date'].unique(), columns=['date'])
min_day = np.min(dailychart['date'])
max_day = np.max(dailychart['date'])
df_date['idx'] = df_date.index

dailychart = pd.merge(dailychart, df_date, on='date')

print("종목정보 가져오는 중...")
stockitems_original = pd.read_csv('stockItems.csv')   #종목정보 가져오기
stockitems_in_chart = dailychart['code'].unique()
stockitems = stockitems_original.loc[stockitems_original['code'].isin(stockitems_in_chart)]

print("지수정보 가져오는 중...")
dailychart_index = pd.read_csv('dailychart_index.csv')   #종목정보 가져오기
dailychart_index['date'] = pd.to_datetime(dailychart_index['date'], format='%Y%m%d')
dailychart_index = pd.merge(dailychart_index, df_date, on='date')
dailychart_index = dailychart_index.loc[(min_day <= dailychart_index['date']) & ( dailychart_index['date'] <= max_day)].reset_index(drop=True)
dailychart_KOSPI = dailychart_index.loc[dailychart_index['section'] == 'KOSPI'].reset_index(drop=True)
dailychart_KOSDAQ = dailychart_index.loc[dailychart_index['section'] == 'KOSDAQ'].reset_index(drop=True)

# 각 종목별 시작일에 날짜 인덱스를 추가
tmp_df = pd.merge(dailychart.groupby('code', as_index=False)['date'].min(), df_date, on='date' )

# 종목 데이터에 종목별 시작일과 시작날짜 인덱스를 조인
stockitems = pd.merge(stockitems_original, tmp_df, on='code')
stockitems = stockitems.rename(columns = {'date':'firstdate', 'idx':'idx_firstdate'})

pd.options.display.float_format = '{:.0f}'.format

print('데이터를 불러왔습니다.')


종목 차트정보 가져오는 중...
종목정보 가져오는 중...
지수정보 가져오는 중...
데이터를 불러왔습니다.


In [3]:
# 벡터연산 기반 재투자 시뮬레이션

# 데이터 1: 매매 로그
column_tradelog = ['idx_date', 'idx_agent', 'idx_stocktrade', 'code', 'buy_or_sell', 'price', 'amount', 'period', 'rate']
tradelog = pd.DataFrame(columns= column_tradelog)  # 전체 tradelog 데이터
tmp_tradelog = pd.DataFrame(columns= column_tradelog)  # 당일 하루치 tradelog 데이터

np.random.seed(0)

# 데이터 2 : 에이전트의 보유 주식 현황
column_stocktrade = ['idx', 'idx_agent', 'code', 'idx_buydate', 'buy_price', 'amount', 'p_price', 'p_amount', 'evaluation']
df_stocktrade = pd.DataFrame(columns = column_stocktrade)

# 데이터 3 : 에이전트 리스트
n_test = 10000
column_agent = ['idx_init_date', 'init_capital', 'volume', 'n_times', 'balance', 't_evaluation', 'rate']  # 초기 투자금, 거래금액, 거래횟수, 잔고(현금), 평가금액(현금+투자금), 수익률

arr_idx = np.arange(0, n_test)
df_agent_list = pd.DataFrame(data = arr_idx, columns = ['idx_agent'])
df_agent_list[column_agent] = 0
df_agent_list[['init_capital','balance','t_evaluation']] = 100000000
df_agent_list['rate'] = df_agent_list['t_evaluation'] / df_agent_list['init_capital']
df_agent_list['idx_init_date'] = np.random.randint(0, len(df_date)-252, size = n_test)
df_agent = pd.DataFrame(columns = ['idx_agent'] + column_agent)


# 데이터 4 : 에이전트 일일단위 총 평가금액 기록

column_dailyeval = ['idx_agent', 'idx_date', 'balance', 't_evaluation']  # 에이전트, 날짜, 잔고(현금), 평가금액(현금+투자금)
df_dailyeval = pd.DataFrame(columns = column_dailyeval)


In [9]:
df_dailyeval

,idx_agent,idx_date,balance,t_evaluation


In [7]:
# 투자기간 설정


print('투자 시뮬레이션을 시작합니다.')
start = time.time()
num = 0

i = 0

for date in df_date['idx']:
    if i == 100:
        break
    print(date)
    # 1. 시세정보 업데이트
    
    # 기준일까지의 시세정보 추출
    #tmp_dailychart = dailychart.loc[dailychart['idx'] < date]
    #tmp_dailychart_KOSPI = dailychart_KOSPI.loc[dailychart_KOSPI['idx'] < date]
    #tmp_dailychart_KOSDAQ = dailychart_KOSDAQ.loc[dailychart_KOSDAQ['idx'] < date]
    
    # 기준일 당시 상장된 주식 리스트 추출
    tmp_stockitems = stockitems.loc[stockitems['idx_firstdate'] <= date].reset_index(drop=True)

    # 기준일 당시 매매 주가 및 평가금액 갱신
    today_dailychart = dailychart.loc[dailychart['idx'] == date]
    
    if len(df_stocktrade) > 0:
        df_stocktrade['p_price'] = pd.merge(df_stocktrade, today_dailychart, how='left', on='code')['open']
        df_stocktrade['evaluation'] = df_stocktrade['p_price'] * df_stocktrade['p_amount']
    
    
        # 기존 투자자에 대한 일일 에이전트 평가금액 추적 데이터 신규 등록
        tmp_df_evaluation_by_agent = df_stocktrade.groupby('idx_agent')['evaluation'].sum()
        tmp_df_dailyeval = pd.merge(tmp_df_dailyeval, tmp_df_evaluation_by_agent, how='left', on='idx_agent') 
        tmp_df_dailyeval['evaluation'] = tmp_df_dailyeval['evaluation'].fillna(0)
        tmp_df_dailyeval['t_evaluation'] = tmp_df_dailyeval['balance'] + tmp_df_dailyeval['evaluation']
        tmp_df_dailyeval = tmp_df_dailyeval[['idx_agent', 'idx_date', 'balance','t_evaluation']]

    # 투자 시작일이 된 투자자(신규 투자자)들 dataframe 추출
    tmp_df_agent = df_agent_list.loc[df_agent_list['idx_init_date'] == date].reset_index(drop=True)
    df_agent = df_agent.append(tmp_df_agent)
    df_agent = df_agent.reset_index(drop=True)

    
    # 신규 투자자에 대한 일일 에이전트 평가금액 추적 데이터 신규 등록    
    
    tmp_df_dailyeval = tmp_df_dailyeval.append(tmp_df_agent[['idx_agent','balance','t_evaluation']]) if len(df_stocktrade)>0 else tmp_df_agent[['idx_agent','balance','t_evaluation']]
    tmp_df_dailyeval  = tmp_df_dailyeval.reset_index(drop=True)    
    #if len(df_stocktrade) > 0:
        #tmp_df_dailyeval[['idx_agent', 'balance', 't_evaluation']] = tmp_df_dailyeval.append(tmp_df_agent[['idx_agent','balance','t_evaluation']])
        #tmp_df_dailyeval = tmp_df_dailyeval.append(tmp_df_agent[['idx_agent','balance','t_evaluation']])
    #else :
        #tmp_df_dailyeval = tmp_df_agent[['idx_agent','balance','t_evaluation']]
    
    tmp_df_dailyeval['idx_date'] = date
    print(tmp_df_dailyeval)

    # 2. 매도 페이즈
    #2-1 stocktrade 테이블에서 매도할 거래 인덱스와 매도 비율을 추출(완전 랜덤으로)
    # 각 종목별로 확률에 의한 완전 랜덤 매도
    selection_coef = 0.02    # 매일 2% 확률로 매도 결정
    sell_index = np.where(np.random.rand(len(df_stocktrade)) < selection_coef )[0]   # selection_coef 확률로 매도할 매매기록 인덱스 선택
    sell_ratio = np.random.rand(len(sell_index))  # 매도 비율 (최초 매수수량을 기준으로)
    
    #2-2 stocktrade에서 매도할 거래 인덱스에 대해 매도 비율만큼 매도 실행
    tmp_df_stocktrade = df_stocktrade.iloc[sell_index]
    tmp_df_stocktrade['sell_amount'] = np.around(tmp_df_stocktrade['p_amount'] * sell_ratio)
    tmp_df_stocktrade.loc[tmp_df_stocktrade['p_amount'] < tmp_df_stocktrade['sell_amount'], 'sell_amount'] = tmp_df_stocktrade['p_amount']
    tmp_df_stocktrade['sell_evaluation'] = tmp_df_stocktrade['p_price'] * tmp_df_stocktrade['sell_amount']
    tmp_df_stocktrade['p_amount'] -= tmp_df_stocktrade['sell_amount']
    
    df_stocktrade.iloc[tmp_df_stocktrade.index] = tmp_df_stocktrade
    #df_stocktrade = df_stocktrade.loc[df_stocktrade['p_amount'] > 0]
    
    
    #2-3  df_agent에 balance 업데이트
    tmp_df_sell_eval = tmp_df_stocktrade.groupby('idx_agent')['sell_evaluation'].sum()
    if len(tmp_df_sell_eval) > 0 :
        #df_agent.loc[df_agent['idx_agent'].isin(sell_index), 'balance'] += tmp_df_sell_eval.loc[tmp_df_buy_eval['idx_agent'].to_numpy(), 'sell_evalutation']
        df_agent.loc[df_agent['idx_agent'].isin(sell_index), 'balance'] += tmp_df_sell_eval
    
    print(tmp_df_stocktrade)
    #2-3 매도 결과를 일일 에이전트 평가금액에 업데이트
    #tmp_df_buy_eval = tmp_df_stocktrade.groupby('idx_agent')['sell_evaluation'].sum()
    #tmp_df_dailyeval = pd.merge(tmp_df_dailyeval, tmp_df_buy_eval, how='left', on='idx_agent')
    #tmp_df_dailyeval['sell_evaluation'] = tmp_df_dailyeval['sell_evaluation'].fillna(0)
    #tmp_df_dailyeval['balance'] += tmp_df_dailyeval['sell_evaluation']
    #tmp_df_dailyeval['t_evaluation'] -= tmp_df_dailyeval['sell_evaluation']
    #tmp_df_dailyeval = tmp_df_dailyeval[['idx_agent', 'idx_date', 'balance','t_evaluation']]

    
    #2-4  tradelog 기록
    tmp_tradelog = tmp_df_stocktrade[['idx_agent', 'code']]
    tmp_tradelog['idx_date'] = date
    tmp_tradelog[['idx_stocktrade', 'price', 'amount']] = tmp_df_stocktrade[['idx', 'p_price', 'p_amount']]
    tmp_tradelog['buy_or_sell'] = 0
    tmp_tradelog['period'] = date - tmp_df_stocktrade['idx_buydate']
    tmp_tradelog['rate'] = (tmp_df_stocktrade['p_price'] / tmp_df_stocktrade['buy_price']) - 1
    #column_tradelog = ['idx_date', 'idx_agent', 'idx_stocktrade', 'code', 'buy_or_sell', 'price', 'amount', 'period', 'rate']
    #column_stocktrade = ['idx', 'idx_agent', 'code', 'idx_buydate', 'buy_price', 'amount', 'p_price', 'p_amount', 'evaluation']
    tradelog = tradelog.append(tmp_tradelog)
                 
                 
    # 4. 매수 페이즈
    
    # 4-1 매수할 에이전트 및 종목 선정
    buy_index = np.where(np.random.rand(len(df_agent)) >= 0.5 )[0]
    buy_ratio = np.random.rand(len(buy_index))  # 매수 비율 (전체 평가금액을 기준으로)
    selected_stockitem = np.random.choice(today_dailychart['code'], len(buy_index))
                 
    tmp_df_agent = df_agent.iloc[buy_index]
    tmp_df_agent['code'] = selected_stockitem
    tmp_df_agent = pd.merge(tmp_df_agent, today_dailychart, on='code')
               
    # 4-2 매수수량 및 금액 계산
    tmp_df_agent['buy_price'] = np.random.randint(tmp_df_agent['low'], tmp_df_agent['high']+1)
    tmp_df_agent['buy_capital'] = np.around((tmp_df_agent['t_evaluation'] * buy_ratio).to_numpy(dtype=float))
    #아래코드는 오류남(위의 코드처럼 수정). series 타입은 np.around가 안되며, to_numpy()해도 dtype 지정안하면 원소타입이 object가 되어 역시 np.around가 안됨
    #tmp_df_agent['buy_capital'] =  np.around(tmp_df_agent['t_evaluation'] * buy_ratio) 
    tmp_df_agent.loc[tmp_df_agent['buy_capital'] > tmp_df_agent['balance'], 'buy_capital'] = tmp_df_agent.loc[tmp_df_agent['buy_capital'] > tmp_df_agent['balance'], 'balance']

    tmp_df_agent['amount'] = np.trunc(tmp_df_agent['buy_capital'] /  tmp_df_agent['buy_price'])
    tmp_df_agent = tmp_df_agent.loc[tmp_df_agent['amount'] > 0]
    
    # 4-3 매수기록 stocktrade 추가
    tmp_df_stocktrade = pd.DataFrame(columns = column_stocktrade)
    begin_index = np.max(df_stocktrade['idx'])+1 if len(df_stocktrade) > 0 else 0
    tmp_df_stocktrade['idx'] = np.arange(begin_index, begin_index+len(tmp_df_agent) )
    tmp_df_stocktrade['idx_buydate'] = date                 
    tmp_df_stocktrade[['idx_agent', 'code', 'buy_price', 'amount']] = tmp_df_agent[['idx_agent', 'code', 'buy_price', 'amount']]
    tmp_df_stocktrade['p_amount'] = tmp_df_stocktrade['amount']
    tmp_df_stocktrade['evaluation'] = tmp_df_stocktrade['buy_price'] * tmp_df_stocktrade['amount']

    # column_stocktrade = ['idx', 'idx_agent', 'code', 'idx_buydate', 'buy_price', 'amount', 'p_price', 'p_amount', 'evaluation']
    
    #4-3  df_agent에 balance 업데이트
    tmp_df_buy_eval = tmp_df_stocktrade.groupby('idx_agent')['evaluation'].sum()
    if len(tmp_df_buy_eval) > 0 :
        df_agent.loc[df_agent['idx_agent'].isin(tmp_df_buy_eval.index), 'balance'] -= tmp_df_buy_eval.values

    # 4-5  tradelog 기록
    tmp_tradelog = tmp_df_stocktrade[['idx_agent', 'code']]
    tmp_tradelog['idx_date'] = date
    tmp_tradelog[['idx_stocktrade', 'price', 'amount']] = tmp_df_stocktrade[['idx', 'buy_price', 'amount']]
    tmp_tradelog['buy_or_sell'] = 1
    
    tradelog = tradelog.append(tmp_tradelog)
    

        
    # 5. 일일 에이전트 평가금액에 업데이트 
    #tmp_df_buy_eval = tmp_df_stocktrade.groupby('idx_agent')['evaluation'].sum()
    tmp_df_dailyeval['balance'] = df_agent['balance']
    
    tmp_df_dailyeval = pd.merge(tmp_df_dailyeval, tmp_df_buy_eval, how='left', on='idx_agent')
    tmp_df_dailyeval['evaluation'] = tmp_df_dailyeval['evaluation'].fillna(0)
    
   #tmp_df_dailyeval['t_evaluation'] = tmp_df_dailyeval['balance'] + tmp_df_dailyeval['evaluation']
    tmp_df_dailyeval = tmp_df_dailyeval[['idx_agent', 'idx_date', 'balance','t_evaluation']]             
    
    # 각종 당일 데이터(tmp_~)를 전체 데이터에 추가
    df_stocktrade = df_stocktrade.append(tmp_df_stocktrade)
    df_dailyeval = df_dailyeval.append(tmp_df_dailyeval)
                 
    # 6. 임시 테이블 초기화
    
    print(tmp_df_dailyeval)
    
    i += 1
    
print('time : ', time.time() - start)
print('투자 시뮬레이션 데이터 로그를 저장합니다.')

#tradelog.to_csv('tradelog_reinvest.csv')
print('시뮬레이션 데이터 로그를 저장하였습니다.')

    

투자 시뮬레이션을 시작합니다.
0
   idx_agent    balance  t_evaluation  idx_date
0        515  100000000     100000000         0
1        759  100000000     100000000         0
2       2735  100000000     100000000         0
3       4916  100000000     100000000         0
4       6428  100000000     100000000         0
5       6774  100000000     100000000         0
6       8875  100000000     100000000         0
7       8994  100000000     100000000         0
8       9605  100000000     100000000         0
Empty DataFrame
Columns: [idx, idx_agent, code, idx_buydate, buy_price, amount, p_price, p_amount, evaluation, sell_amount, sell_evaluation]
Index: []
   idx_agent  idx_date    balance  t_evaluation
0        515         0  100000000     100000000
1        759         0   68980470     100000000
2       2735         0   21010068     100000000
3       4916         0  100000000     100000000
4       6428         0  100000000     100000000
5       6774         0   90346195     100000000
6       8875  

TypeError: loop of ufunc does not support argument 0 of type float which has no callable rint method

In [13]:
# 다시 돌릴 때 테이블 초기화

tmp_stockitems = np.nan
today_dailychart = np.nan
tmp_df_evaluation_by_agent = np.nan
tmp_df_dailyeval = np.nan
tmp_df_agent = np.nan
tmp_df_stocktrade = np.nan
tmp_df_sell_eval = np.nan
tmp_tradelog = np.nan
selected_stockitem = np.nan
tmp_df_buy_eval = np.nan

In [9]:
df_agent_list


,idx_agent,idx_init_date,init_capital,volume,n_times,balance,t_evaluation,rate
0,0,684,100000000,0,0,100000000,100000000,1
1,1,559,100000000,0,0,100000000,100000000,1
2,2,1216,100000000,0,0,100000000,100000000,1
3,3,835,100000000,0,0,100000000,100000000,1
4,4,763,100000000,0,0,100000000,100000000,1
...,...,...,...,...,...,...,...,...
9995,9995,1025,100000000,0,0,100000000,100000000,1
9996,9996,873,100000000,0,0,100000000,100000000,1
9997,9997,309,100000000,0,0,100000000,100000000,1
9998,9998,837,100000000,0,0,100000000,100000000,1


In [7]:
df_dailyeval.loc[df_dailyeval['idx_date'] == 3]

,idx_agent,idx_date,balance,t_evaluation
0,515,3,45606815,106323486
1,759,3,6585,95364185
2,2735,3,-22776912,98659108
3,4916,3,65562970,106894870
4,6428,3,10369693,38171962
5,6774,3,90346195,99883864
6,8875,3,-85713568,69589806
7,8994,3,37620,52273540
8,9605,3,11768,65833621
9,4225,3,78990010,100000000


In [11]:
df_stocktrade

,idx,idx_agent,code,idx_buydate,buy_price,amount,p_price,p_amount,evaluation
0,0,759,A052190,0,10645,2914,10600,2914,30888400
1,1,2735,A104200,0,9013,8764,8720,8764,76422080
2,2,6774,A002900,0,665,14517,662,14517,9610254
3,3,8875,A002680,0,1626,35519,1620,35519,57540780
4,4,8994,A033660,0,5005,10672,4970,10672,53039840
5,5,9605,A001067,0,12827,3361,12845,3361,43172045
0,6,759,A012510,1,11027,6255,NaN,6255,68973885
1,7,4916,A115570,1,7382,4665,NaN,4665,34437030
2,8,8875,A101730,1,20757,1467,NaN,1467,30450519
3,9,8994,A086790,1,43708,1065,NaN,1065,46549020


In [10]:
df_stocktrade.groupby('idx_agent')['evaluation'].sum()

idx_agent
759      99862285
2735     76422080
4916     34437030
6774      9610254
8875     87991299
8994     99588860
9605    100048730
Name: evaluation, dtype: int64

,idx_x,idx_agent,code,idx_buydate,buy_price,amount,p_price,p_amount,evaluation,section,...,low,close,vol,value,n_stock,agg_price,foreign_rate,agency_buy,agency_netbuy,idx_y
0,0,759,A052190,0,10645,2914,10400,2914,30305600,KOSDAQ,...,10250,10500,37947,393,6210000,65205,1,0,8656,2
1,1,2735,A104200,0,9013,8764,8780,8764,76947920,KOSDAQ,...,8700,8820,33607,295,13527000,119308,1,4916,539042,2
2,2,6774,A002900,0,665,14517,662,14517,9610254,KOSPI,...,657,657,64200,51,65388000,42959,14,0,92760,2
3,3,8875,A002680,0,1626,35519,1580,35519,56120020,KOSDAQ,...,1580,1600,27595,44,20800000,33280,0,0,-7584,2
4,4,8994,A033660,0,5005,10672,4830,10672,51545760,KOSPI,...,4760,4760,12366,59,57545000,273914,0,-5050,2821352,2
5,5,9605,A001067,0,12827,3361,13007,3361,43716527,KOSPI,...,12966,13495,460,7,71000,958,0,0,-671,2
6,6,759,A012510,1,11027,6255,10400,6255,65052000,KOSPI,...,11000,11350,200880,2260,29672000,336777,5,-10,1661071,2
7,7,4916,A115570,1,7382,4665,8780,4665,40958700,KOSDAQ,...,7010,7010,1770,13,5400000,37854,0,0,283588,2
8,8,8875,A101730,1,20757,1467,662,1467,971154,KOSDAQ,...,20285,20639,48352,1123,7040000,145298,1,-12108,1235317,2
9,9,8994,A086790,1,43708,1065,1580,1065,1682700,KOSPI,...,41800,42600,771248,32860,289894000,12349484,62,-85482,-2044018,2


In [4]:
tmp_df_dailyeval

,idx_agent,idx_date,balance,t_evaluation
0,515,2,100000000,100000000
1,759,2,6585,95364185
2,2735,2,21010068,97957988
3,4916,2,65562970,106521670
4,6428,2,100000000,100000000
5,6774,2,90346195,99956449
6,8875,2,11795587,68886761
7,8994,2,37620,53266080
8,9605,2,11768,65999425
9,4225,2,100000000,100000000


In [6]:
df_agent

,idx_agent,idx_init_date,init_capital,volume,n_times,balance,t_evaluation,rate
0,515,0,100000000,0,0,59045086,100000000,1
1,759,0,100000000,0,0,6585,100000000,1
2,2735,0,100000000,0,0,21010068,100000000,1
3,4916,0,100000000,0,0,65562970,100000000,1
4,6428,0,100000000,0,0,10369693,100000000,1
5,6774,0,100000000,0,0,90346195,100000000,1
6,8875,0,100000000,0,0,11795587,100000000,1
7,8994,0,100000000,0,0,37620,100000000,1
8,9605,0,100000000,0,0,11768,100000000,1
0,4225,1,100000000,0,0,100000000,100000000,1


In [59]:
iter = 0
coef = 0.02

sum = 0.0
a = 0.0  # 확률에 걸린 집단
b = 1.0  # 확률에 걸리지 않은 집단

while sum < 0.99:  # 확률에 걸린 집단의 총합이 0.99를 넘을 때 까지 반복
    a = b * coef
    sum += a 
    b -= a 
    iter += 1
    
print(iter)

228


In [27]:
start = time.time()
today_dailychart = dailychart.loc[dailychart['idx'] == 20]
df_stocktrade['p_price'] = pd.merge(df_stocktrade, today_dailychart, left_on='code')['start']

['idx', 'idx_agent', 'code', 'idx_buydate', 'buy_price', 'amount', 'evaluation']


print('time : ', time.time() - start)

time :  0.004987239837646484


In [28]:
pd.merge(df_stocktrade, today_dailychart, left_on='code')

NameError: name 'df_stocktrade' is not defined

In [ ]:
# 벡터연산 기반 시뮬레이션


column_tradelog = ['code', 'name', 'section', 'idx_buydate', 'buydate','buyprice', 'idx_selldate', 'selldate', 'sellprice', 'period', 'rate','buy_index','sell_index','rate_index']
tradelog = pd.DataFrame(columns= column_tradelog)

n_test = 100000
np.random.seed(0)

print('투자 시뮬레이션을 시작합니다.')
start = time.time()

# 무작위로 투자 종목 선택
tradelog[['code','name', 'section', 'idx_firstdate']] = stockitems.loc[np.random.randint(0, len(stockitems), size=n_test), ['code', 'name', 'section','idx_firstdate']]

# 무작위로 매수일 선택 (해당 종목의 최초 상장일부터 투자기간 마지막날의 전일까지 중 랜덤)
tradelog['idx_buydate'] = tradelog.apply(lambda x: np.random.randint(x['idx_firstdate'], len(df_date)-1) ,axis = 1)
tradelog['buydate'] = df_date.loc[tradelog['idx_buydate'],'date'].to_numpy()
tradelog = pd.merge(tradelog, dailychart[['code','date','low','high']], left_on=['code','buydate'],  right_on=['code','date'], how='left')
tradelog['buyprice'] = np.random.randint(tradelog['low'], tradelog['high']+1)
tradelog = tradelog.drop(columns=['low', 'high'])

tradelog['idx_selldate'] = tradelog['idx_buydate'] + np.random.randint(1, len(df_date) - tradelog['idx_buydate'])
tradelog['selldate'] = df_date.loc[tradelog['idx_selldate'],'date'].to_numpy()
tradelog = pd.merge(tradelog, dailychart[['code','date','low','high']], left_on=['code','selldate'],  right_on=['code','date'], how='left')
tradelog['sellprice'] = np.random.randint(tradelog['low'], tradelog['high']+1)
tradelog = tradelog.drop(columns=['low', 'high'])
tradelog['period'] = tradelog['idx_selldate'] - tradelog['idx_buydate']
tradelog['rate'] = (tradelog['sellprice'] - tradelog['buyprice']) / tradelog['buyprice'] * 100

tradelog['buy_index'] = dailychart_KOSPI.loc[tradelog['idx_buydate'], 'close'].to_numpy()
tradelog['sell_index'] = dailychart_KOSPI.loc[tradelog['idx_selldate'], 'close'].to_numpy()
tradelog_index_KOSDAQ = tradelog.loc[tradelog['section'] == 'KOSDAQ'].index
tradelog.loc[tradelog_index_KOSDAQ, 'buy_index'] = dailychart_KOSDAQ.loc[tradelog.loc[tradelog_index_KOSDAQ,'idx_buydate'], 'close'].to_numpy()
tradelog.loc[tradelog_index_KOSDAQ, 'sell_index'] = dailychart_KOSDAQ.loc[tradelog.loc[tradelog_index_KOSDAQ,'idx_selldate'], 'close'].to_numpy()

tradelog['rate_index'] =  (tradelog['sell_index'] - tradelog['buy_index']) / tradelog['buy_index'] * 100

#불필요한 컬럼 제거
tradelog = tradelog.drop(columns=['date_x','date_y', 'idx_firstdate'])

print('time : ', time.time() - start)
print('투자 시뮬레이션 데이터 로그를 저장합니다.')

tradelog.to_csv('tradelog3.csv')
print('시뮬레이션 데이터 로그를 저장하였습니다.')

    

In [21]:
column_tradelog = ['code', 'name', 'section', 'idx_buydate', 'buydate','buyprice', 'idx_selldate', 'selldate', 'sellprice', 'period', 'rate','buy_index','sell_index','rate_index']
tradelog = pd.DataFrame(columns= column_tradelog)

n_test = 100000
np.random.seed(0)

print('투자 시뮬레이션을 시작합니다.')
start = time.time()

tradelog[['code','name', 'section', 'idx_firstdate']] = stockitems.loc[np.random.randint(0, len(stockitems), size=n_test), ['code', 'name', 'section','idx_firstdate']]
tradelog

투자 시뮬레이션을 시작합니다.


,code,name,section,idx_buydate,buydate,buyprice,idx_selldate,selldate,sellprice,period,rate,buy_index,sell_index,rate_index,idx_firstdate
2732,A222420,쎄노텍,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393
2607,A179900,유티아이,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,922
1653,A015750,성우하이텍,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
835,A132030,KODEX 골드선물(H),ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
763,A098560,TIGER 방송통신,ETF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,A122990,와이솔,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2248,A080720,한국유니온제약,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1121
2857,A272110,케이엔제이,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1426
800,A114090,GKL,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
tradelog['net_rate'] = tradelog['rate'] - tradelog['rate_index']
tradelog.describe()

,idx_buydate,buyprice,idx_selldate,sellprice,period,rate,buy_index,sell_index,rate_index,net_rate
count,100000.000000,1.000000e+05,100000.000000,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,1097.887290,2.151344e+04,1445.829020,2.197456e+04,347.941730,13.702470,1518.146901,1652.946859,10.480117,3.222353
std,520.129257,5.705542e+04,360.038521,5.747723e+04,361.610964,83.898861,813.737106,893.613601,19.361336,82.093481
min,0.000000,8.300000e+01,10.000000,6.200000e+01,1.000000,-99.812236,428.350006,428.350006,-51.680221,-177.369213
25%,684.000000,3.832750e+03,1271.000000,3.803000e+03,66.000000,-15.542792,694.659973,807.849976,-1.532391,-24.633075
50%,1199.000000,8.860000e+03,1575.000000,8.901000e+03,218.000000,0.288302,1888.300049,1879.125000,5.823155,-4.086425
75%,1561.000000,1.699675e+04,1727.000000,1.802025e+04,522.000000,19.725553,2157.179932,2349.550049,21.109935,9.956889
max,1791.000000,1.768173e+06,1792.000000,1.662114e+06,1788.000000,5226.899128,3208.989990,3208.989990,136.698963,5214.251084


In [6]:
tradelog_minrate = tradelog.sort_values('rate').head(10)
tradelog_maxrate = tradelog.sort_values('rate', ascending=False).head(10)